In [1]:
# grader-required-cell

import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img

In [ ]:
!wget --no-check-certificate \https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \-O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [10]:
import os
import cv2
import numpy as np

def parse_data_from_input(dataset_dir):
    # Define the path to your dataset directory containing subdirectories for each class

    # Initialize empty lists to store images and labels
    images = []
    labels = []

    # Create a list of class labels based on subdirectory names
    class_labels = os.listdir(dataset_dir)

    # Loop through each subdirectory (class)
    for label, class_name in enumerate(class_labels):
        class_dir = os.path.join(dataset_dir, class_name)
        count = 1 
        # Loop through all files in the class directory
        for filename in os.listdir(class_dir):
            if filename.endswith((".jpg", ".png")):  # Adjust file extensions as needed
                new_path = os.path.join(class_dir, filename)
                 # Specify the old and new file names
                # Generate a unique filename by adding a count to the original filename
#                 base_filename, file_extension = os.path.splitext(filename)
#                 new_filename = f"{class_name}_{count}{file_extension}"
#                 new_path = os.path.join(class_dir, new_filename)
                
                # Increment the count to ensure the next file has a unique name
                count += 1

                # Rename the file
#                 os.rename(file_path, new_path)
                # Load the image using OpenCV
                image = cv2.imread(new_path)
                # If you want to save the grayscale image to a file
                cv2.imwrite(new_path, image)

                # Resize the image to a consistent size if needed
                image = cv2.resize(image, (224, 224))

                # Append the image and its corresponding label to the lists
                images.append(image)
                labels.append(label)

    # Convert the lists to NumPy arrays
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

    # You can then use 'images' and 'labels' in your machine learning model

In [11]:
TRAINING_FILE = r'train/'
VALIDATION_FILE = r'test/'

In [12]:
# grader-required-cell

# Test your function
training_images, training_labels = parse_data_from_input(TRAINING_FILE)
validation_images, validation_labels = parse_data_from_input(VALIDATION_FILE)

print(f"Training images has shape: {training_images.shape} and dtype: {training_images.dtype}")
print(f"Training labels has shape: {training_labels.shape} and dtype: {training_labels.dtype}")
print(f"Validation images has shape: {validation_images.shape} and dtype: {validation_images.dtype}")
print(f"Validation labels has shape: {validation_labels.shape} and dtype: {validation_labels.dtype}")

Training images has shape: (4682, 224, 224, 3) and dtype: uint8
Training labels has shape: (4682,) and dtype: int32
Validation images has shape: (355, 224, 224, 3) and dtype: uint8
Validation labels has shape: (355,) and dtype: int32


In [108]:
print(training_labels[1500])

1


In [13]:
# grader-required-cell

# GRADED FUNCTION: train_val_generators
def train_val_generators(training_images, training_labels, validation_images, validation_labels):
# def train_val_generators(training_images, training_labels):
  train_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
      )


  # Pass in the appropriate arguments to the flow method
#   train_generator = train_datagen.flow(x=training_images,
#                                        y=training_labels,
#                                        batch_size=126,
# )
  train_generator = train_datagen.flow_from_directory(
	  TRAINING_FILE,
	  target_size=(224,224),
	  class_mode='categorical',
    batch_size=126
)
  validation_datagen = ImageDataGenerator(rescale = 1./255.)

  # Pass in the appropriate arguments to the flow method
  # validation_generator = validation_datagen.flow(x=validation_images,
  #                                                y=validation_labels,
  #                                                batch_size=126)
  validation_generator = validation_datagen.flow_from_directory(
	  VALIDATION_FILE,
	  target_size=(224,224),
	  class_mode='categorical',
    batch_size=126
  )
  ### END CODE HERE

  return train_generator, validation_generator

In [14]:
train_generator, validation_generator = train_val_generators(training_images, training_labels, validation_images, validation_labels)
# train_generator = train_val_generators(training_images, training_labels)

# print(f"Images of training generator have shape: {train_generator.x.shape}")
# print(f"Labels of training generator have shape: {train_generator.y.shape}")
# print(f"Images of validation generator have shape: {validation_generator.x.shape}")
# print(f"Labels of validation generator have shape: {validation_generator.y.shape}")

Found 4682 images belonging to 4 classes.
Found 356 images belonging to 4 classes.


In [190]:
# import urllib.request
# from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.optimizers import RMSprop
# inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
# urllib.request.urlretrieve(inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
# local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# pre_trained_model = InceptionV3(input_shape=(224, 224, 3),include_top=False,weights=None)
# pre_trained_model.load_weights(local_weights_file)

# for layer in pre_trained_model.layers:
#     layer.trainable = False

# last_layer = pre_trained_model.get_layer('mixed7')
# last_output = last_layer.output

In [15]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
base_model=MobileNetV2(input_shape=(224,224,3),weights='imagenet',include_top=False) 

# add Fully-Connected Layers to Model
x = base_model.output
x = layers.GlobalAveragePooling2D()(base_model.output)

In [20]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            # Check accuracy
            if (logs.get('accuracy') > 0.97 and logs.get('val_accuracy') > 0.97):
                # Stop if threshold is met
                print("\n\n\t=====================================")
                print("\t|| accuracy and val_accuracy > 80% ||")
                print("\t=====================================\n")
                self.model.stop_training = True

callbacks = myCallback()

In [21]:

x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(16, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(4, activation='softmax')(x)

model = Model(base_model.input, x)

model.compile(optimizer = RMSprop(learning_rate=0.0001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])
model.fit(
        train_generator,
        epochs=10,
        verbose=1,
        validation_data=validation_generator,
        callbacks=[callbacks]
    )

Epoch 1/10
38/38 [==============================] - 302s 7s/step - loss: 1.2743 - accuracy: 0.7426 - val_loss: 1.0119 - val_accuracy: 0.9860
Epoch 2/10
38/38 [==============================] - 299s 8s/step - loss: 0.8205 - accuracy: 0.8633 - val_loss: 0.4238 - val_accuracy: 0.9860
Epoch 3/10
38/38 [==============================] - 288s 8s/step - loss: 0.4103 - accuracy: 0.9199 - val_loss: 0.1660 - val_accuracy: 0.9860
Epoch 4/10
38/38 [==============================] - 281s 7s/step - loss: 0.2200 - accuracy: 0.9513 - val_loss: 0.0748 - val_accuracy: 0.9888
Epoch 5/10
38/38 [==============================] - 283s 7s/step - loss: 0.1343 - accuracy: 0.9622 - val_loss: 0.0216 - val_accuracy: 0.9972
Epoch 6/10
38/38 [==============================] - 284s 7s/step - loss: 0.1174 - accuracy: 0.9643 - val_loss: 0.0274 - val_accuracy: 0.9944
Epoch 7/10
38/38 [==============================] - ETA: 0s - loss: 0.0928 - accuracy: 0.9712

	|| accuracy and val_accuracy > 80% ||

38/38 [============

In [111]:
# def create_model():

#   ### START CODE HERE

#   # Define the model
#   # Use no more than 2 Conv2D and 2 MaxPooling2D
#   model = tf.keras.models.Sequential([
#       tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
#       tf.keras.layers.MaxPooling2D(2,2),
#       tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
#       tf.keras.layers.MaxPooling2D(2, 2),   
#       tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
#       tf.keras.layers.MaxPooling2D(2, 2),
#       tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#       tf.keras.layers.MaxPooling2D(2, 2),
#       tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#       tf.keras.layers.MaxPooling2D(2, 2),
#       tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
#       tf.keras.layers.MaxPooling2D(2, 2),
#       # Flatten the results to feed into a DNN
#       tf.keras.layers.Flatten(),
#       # 512 neuron hidden layer
#       tf.keras.layers.Dense(1028, activation='relu'),
#       tf.keras.layers.Dense(512, activation='relu'),
#       tf.keras.layers.Dense(512, activation='relu'),
#       tf.keras.layers.Dense(256, activation='relu'),
#       tf.keras.layers.Dense(128, activation='relu'),
#       tf.keras.layers.Dense(64, activation='relu'),
#       tf.keras.layers.Dense(32, activation='relu'),
#       tf.keras.layers.Dense(4, activation='softmax')
#   ])


#   model.compile(optimizer = 'rmsprop',
#                 loss = 'categorical_crossentropy',
#                 metrics=['accuracy'])

#   ### END CODE HERE

#   return model

In [94]:
# class myCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') > 0.9998 and logs.get('val_accuracy') > 0.9998 and logs.get('loss') < 0.001 and logs.get('val_loss') > 0.001):
#             self.model.stop_training = True

# calbak = myCallback()

In [207]:
# # Save your model
# model = create_model()

# # Train your model
# history = model.fit(train_generator,
#                     validation_data=validation_generator,
#                     epochs=30,
#                     verbose = 1)


In [1]:
# #-----------------------------------------------------------
# # Retrieve a list of list results on training and test data
# # sets for each training epoch
# #-----------------------------------------------------------
# acc=model.model['accuracy']
# val_acc=model.model['val_accuracy']
# loss=model.model['loss']
# val_loss=model.model['val_loss']

# epochs=range(len(acc)) # Get number of epochs

# #------------------------------------------------
# # Plot training and validation accuracy per epoch
# #------------------------------------------------
# plt.plot(epochs, acc, 'r', "Training Accuracy")
# plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
# plt.title('Training and validation accuracy')
# plt.show()
# print("")

# #------------------------------------------------
# # Plot training and validation loss per epoch
# #------------------------------------------------
# plt.plot(epochs, loss, 'r', "Training Loss")
# plt.plot(epochs, val_loss, 'b', "Validation Loss")
# plt.show()

In [25]:
model.save("15Desember2023-2.h5")

In [38]:
# # # Load the model
from keras.models import load_model  # TensorFlow is required for Keras to work
import cv2  # Install opencv-python
import numpy as np
model = load_model("15Desember2023.h5", compile=False)

In [22]:

## NOTE: If you are using Safari and this cell throws an error,
## please skip this block and run the next one instead.

import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

 
  # predicting images
path=r'test\Down-Correct\9_59_8_25881_png.rf.c9feedfd7b97964360230b5b58942527.jpg'
img=load_img(path, target_size=(224, 224))
  
x=img_to_array(img)
x /= 255
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
  
classes = model.predict(images, batch_size=16)
print("Down-Correct")  
print(classes)


1/1 [==============================] - 2s 2s/step
Down-Correct
[[9.9980479e-01 4.1273775e-05 1.1687775e-04 3.7082104e-05]]


In [23]:

## NOTE: If you are using Safari and this cell throws an error,
## please skip this block and run the next one instead.

import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

 
  # predicting images
path=r'test\Down-Wrong\36_7_380 - Copy.jpg'
img=load_img(path, target_size=(224, 224))
  
x=img_to_array(img)
x /= 255
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
  
classes = model.predict(images, batch_size=16)
print("Down-Wrong")  
print(classes)


1/1 [==============================] - 0s 54ms/step
Down-Wrong
[[7.0448586e-05 9.9784565e-01 2.4517823e-05 2.0594753e-03]]


In [24]:

## NOTE: If you are using Safari and this cell throws an error,
## please skip this block and run the next one instead.

import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

 
  # predicting images
path=r'test\Up-Correct\73_2464_png.rf.2f909fcd964c70cb7497abf4e5501142.jpg'
img=load_img(path, target_size=(224, 224))
  
x=img_to_array(img)
x /= 255
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
  
classes = model.predict(images, batch_size=16)
print("Up-Correct")   
print(classes[0])


1/1 [==============================] - 0s 50ms/step
Up-Correct
[6.8621034e-06 2.5139449e-04 9.9973899e-01 2.7283929e-06]


In [25]:

## NOTE: If you are using Safari and this cell throws an error,
## please skip this block and run the next one instead.

import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

 
  # predicting images
path=r'test\Up-Wrong\74_21_390.jpg'
img=load_img(path, target_size=(224, 224))
  
x=img_to_array(img)
x /= 255
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
  
classes = model.predict(images, batch_size=16)
print("Up-Wrong")  
print(classes[0])


1/1 [==============================] - 0s 66ms/step
Up-Wrong
[1.0375864e-03 1.2643187e-02 4.4660796e-05 9.8627454e-01]


In [28]:
## NOTE: If you are using Safari and this cell throws an error,
## please skip this block and run the next one instead.

import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

 
  # predicting images
path=r'Others\cat.jpeg'
img=load_img(path, target_size=(224, 224))
  
x=img_to_array(img)
x /= 255
x=np.expand_dims(x, axis=0)
images = np.vstack([x])
  
classes = model.predict(images, batch_size=16)
print("Down-Wrong")  
print(classes)

1/1 [==============================] - 0s 47ms/step
Down-Wrong
[[9.9980468e-01 4.1279913e-05 1.1689356e-04 3.7087546e-05]]


In [103]:
# import tensorflow as tf
# from tensorflow.keras.applications import MobileNetV2
# model.save('Capstone Project_WOAI_model.h5')

In [205]:
# # Load the model
# model = load_model("keras_model.h5", compile=False)

# # Load the labels
# class_names = open("labels.txt", "r").readlines()

In [202]:
# from keras.models import load_model  # TensorFlow is required for Keras to work
# import cv2  # Install opencv-python
# import numpy as np

# # Disable scientific notation for clarity
# np.set_printoptions(suppress=True)

# path=r'test\Down-Correct\9_59_8_25881_png.rf.c9feedfd7b97964360230b5b58942527.jpg'
# image=load_img(path, target_size=(224, 224))

# image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

# # Normalize the image array
# image = (image / 127.5) - 1

# # Predicts the model
# prediction = model.predict(image)
# index = np.argmax(prediction)
# class_name = class_names[index]
# confidence_score = prediction[0][index]

# # Print prediction and confidence score
# print("Class:", class_name[2:], end="")
# print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")


In [201]:
# from keras.models import load_model  # TensorFlow is required for Keras to work
# import cv2  # Install opencv-python
# import numpy as np

# # Disable scientific notation for clarity
# np.set_printoptions(suppress=True)

# path=r'test\Down-Wrong\image2_432.jpg'
# image=load_img(path, target_size=(224, 224))

# image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

# # Normalize the image array
# image = (image / 127.5) - 1

# # Predicts the model
# prediction = model.predict(image)
# index = np.argmax(prediction)
# class_name = class_names[index]
# confidence_score = prediction[0][index]

# # Print prediction and confidence score
# print("Class:", class_name[2:], end="")
# print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")


In [203]:
# from keras.models import load_model  # TensorFlow is required for Keras to work
# import cv2  # Install opencv-python
# import numpy as np

# # Disable scientific notation for clarity
# np.set_printoptions(suppress=True)

# path=r'test\Up-Correct\6068_png.rf.31490652a258e3f4f98e89015b326a31.jpg'
# image=load_img(path, target_size=(224, 224))

# image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

# # Normalize the image array
# image = (image / 127.5) - 1

# # Predicts the model
# prediction = model.predict(image)
# index = np.argmax(prediction)
# class_name = class_names[index]
# confidence_score = prediction[0][index]

# # Print prediction and confidence score
# print("Class:", class_name[2:], end="")
# print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")


In [204]:
# from keras.models import load_model  # TensorFlow is required for Keras to work
# import cv2  # Install opencv-python
# import numpy as np

# # Disable scientific notation for clarity
# np.set_printoptions(suppress=True)

# path=r'test\Up-Wrong\image8_882.jpg'
# image=load_img(path, target_size=(224, 224))

# image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

# # Normalize the image array
# image = (image / 127.5) - 1

# # Predicts the model
# prediction = model.predict(image)
# index = np.argmax(prediction)
# class_name = class_names[index]
# confidence_score = prediction[0][index]

# # Print prediction and confidence score
# print("Class:", class_name[2:], end="")
# print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")


In [44]:
import tensorflow as tf

# Load the HDF5 model
model_path = '15Desember2023-2.h5'
model = tf.keras.models.load_model(model_path)

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
tflite_path = '15Desember2023-2.tflite'
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f'Model successfully converted to TensorFlow Lite format: {tflite_path}')


INFO:tensorflow:Assets written to: C:\Users\mohsu\AppData\Local\Temp\tmp1keor4ut\assets


INFO:tensorflow:Assets written to: C:\Users\mohsu\AppData\Local\Temp\tmp1keor4ut\assets


Model successfully converted to TensorFlow Lite format: 15Desember2023-2.tflite
